In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from evo import *

params = {
    'd_model': 512,
    'nhead': 8,
    'dim_feedforward': 2048,
    'dropout': 0.1,
    'encoder_layers': 3,
    'decoder_layers': 3,
    'max_len': 64
}

evolver = Evolver(**params)
ps_evolver = PointerStyleEvolver(**params)

In [ ]:
from data import *
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_loader = supervised_loader(
    path='data/ud/ud_train_3.0.jsonl',
    max_len=10,
    tokenizer=tokenizer,
    batch_size=4,
    cache_prefix=None,
    all_tokens=True,
    limit=20,
    sampler=StratifiedInfiniteSampler
)

eval_loader = unsupervised_loader(
    path='data/toy/toy.jsonl',
    max_len=10,
    tokenizer=tokenizer,
    batch_size=4,
    sampler=StratifiedInfiniteSampler
)

In [ ]:
from run import apply_edits

traj_input_ids, _, traj_edit_tgts, _ = next(iter(train_loader))

apply_edits(traj_input_ids[:, 0], tuple(map(lambda x: x[:, 0], traj_edit_tgts)))

In [ ]:
from torch.optim import AdamW

kwargs = {
    'train_loader': train_loader,
    'eval_loader': eval_loader,
    'train_steps': 1,
    'eval_steps': 2,
    'grad_accum_steps': 1,
    'clip_gradients': False,
    'checkpoint_at': 20,
    'eval_at': 1
}

print('STARTING REGULAR EVOLVER')
train_evolver(evolver, AdamW(evolver.parameters(), lr=3e-4), None, **kwargs)

print('STARTING PS EVOLVER')
train_evolver(ps_evolver, AdamW(ps_evolver.parameters(), lr=3e-4), None, **kwargs)

## multihead pointer

In [ ]:
import torch
from trans import MultiheadPointer

pointer = MultiheadPointer(512, 8)

mem = torch.randn(3, 10, 512)
tgt = torch.randn(3, 5, 512)
src_pad_mask = torch.full((3, 10), True)
src_pad_mask[:, :7] = False

idx_weights = pointer(tgt, mem, key_padding_mask=src_pad_mask)
idx_weights

## regressions

```
python evo.py --config=configs/toy/sup-toy.json
python evo.py --config=configs/toy/sup-toy-epoch.json
python evo.py --config=configs/toy/ps-sup-toy.json
!python evo.py --config=configs/toy/ps-unsup-toy.json
python evo.py --config=configs/toy/noshare-sup-toy-e1d1.json
python evo.py --config=configs/toy/den-toy.json
```

In [ ]:
!python evo.py --config=configs/toy/sup-toy.json --local

In [ ]:
!python evo.py --config=configs/toy/sup-toy-epoch.json --local

In [ ]:
!python evo.py --config=configs/toy/ps-unsup-toy.json

In [ ]:
!python evo.py --config=configs/toy/ps-sup-toy.json

In [ ]:
!python evo.py --config=configs/toy/den-toy.json

In [4]:
!python evo.py --config=configs/toy/ar-d-toy.json

INFO:train:using <class 'data.StratifiedInfiniteSampler'> in loader
INFO:train:using ar loaders
INFO:data:tokenizing sequence dataset...
INFO:data:done in 0.00 seconds!
tokenizing trajectories: 100%|██████████████████| 4/4 [00:00<00:00, 2262.60it/s]
INFO:train:starting run for 500 steps
INFO:train:eval every 20 steps
INFO:train:checkpoint every 500 steps
INFO:train:using <class '__main__.Transformer'>
INFO:train:starting new run
wandb: Currently logged in as: tjbai. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.3
wandb: Run data is saved locally in /Users/bai/argo/projects/ddm/evolver/wandb/run-20240821_221147-j1dborig
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ar-d-toy_20240821_221146
wandb: ⭐️ View project at https://wandb.ai/tjbai/evolver
wandb: 🚀 View run at https://wandb.ai/tjbai/evolver/runs/j1dborig
100%|█████

In [ ]:
from data import unsupervised_loader, StratifiedInfiniteSampler
from transformers import BertTokenizer

loader = unsupervised_loader(
    'data/toy/toy.jsonl',
    max_len=10,
    tokenizer=BertTokenizer.from_pretrained('bert-base-uncased'),
    batch_size=2,
    sampler=StratifiedInfiniteSampler
)

In [ ]:
for batch in loader:
    print(batch[0].shape)
    traj_input_ids = batch[0]
    break

input_ids = traj_input_ids[:, 0]
output_ids = traj_input_ids[:, 1]

In [ ]:
B, N = input_ids.shape